In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

np.random.seed(100)
tf.random.set_seed(100)

# 데이터를 DataFrame 형태로 불러 옵니다.
df = pd.read_csv("C:/Users/gotsl/Desktop/data_Advertising.csv")

# DataFrame 데이터 샘플 5개를 출력합니다.
print('원본 데이터 샘플 :')
print(df.head(),'\n')

# 의미없는 변수는 삭제합니다.
df = df.drop(columns=['Unnamed: 0'])

"""
1. Sales 변수는 label 데이터로 Y에 저장하고 나머진 X에 저장합니다.
"""
X = df.drop(columns=['Sales'])
Y = df['Sales']

train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.3)

"""
2. 학습용 데이터를 tf.data.Dataset 형태로 변환합니다.
   from_tensor_slices 함수를 사용하여 변환하고 batch_size는 5로 설정합니다.
"""
train_ds = tf.data.Dataset.from_tensor_slices((train_X.values, train_Y))
train_ds = train_ds.shuffle(len(train_X)).batch(batch_size=5)

# 하나의 batch를 뽑아서 feature와 label로 분리합니다.
[(train_features_batch, label_batch)] = train_ds.take(1)

# batch 데이터를 출력합니다.
print('\nFB, TV, Newspaper batch 데이터:\n',train_features_batch)
print('Sales batch 데이터:',label_batch)

원본 데이터 샘플 :
   Unnamed: 0     FB    TV  Newspaper  Sales
0           1  230.1  37.8       69.2   22.1
1           2   44.5  39.3       45.1   10.4
2           3   17.2  45.9       69.3    9.3
3           4  151.5  41.3       58.5   18.5
4           5  180.8  10.8       58.4   12.9 


FB, TV, Newspaper batch 데이터:
 tf.Tensor(
[[296.4  36.3 100.9]
 [228.   37.7  32. ]
 [  5.4  29.9   9.4]
 [ 57.5  32.8  23.5]
 [240.1   7.3   8.7]], shape=(5, 3), dtype=float64)
Sales batch 데이터: tf.Tensor([23.8 21.5  5.3 11.8 13.2], shape=(5,), dtype=float64)


In [10]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# INFO 및 WARNING 로그는 출력되지 않고 ERROR 로그만 출력

np.random.seed(100)
tf.random.set_seed(100)

# 데이터를 DataFrame 형태로 불러 옵니다.
df = pd.read_csv("C:/Users/gotsl/Desktop/data_Advertising.csv")

# DataFrame 데이터 샘플 5개를 출력합니다.
print('원본 데이터 샘플 :')
print(df.head(),'\n')

# 의미없는 변수는 삭제합니다.
df = df.drop(columns=['Unnamed: 0'])

X = df.drop(columns=['Sales'])
Y = df['Sales']

# 학습용 테스트용 데이터로 분리합니다.
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.3)

# Dataset 형태로 변환합니다.
train_ds = tf.data.Dataset.from_tensor_slices((train_X.values, train_Y))
train_ds = train_ds.shuffle(len(train_X)).batch(batch_size=5)

"""
1. keras를 활용하여 신경망 모델을 생성합니다.
   자유롭게 레이어를 쌓고 마지막 레이어에는 노드 수를 1개로 설정합니다.
"""
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, input_shape=(3,)),
    tf.keras.layers.Dense(1)
    ])
# tf.keras.layers.Dense(10, input_shape=(3,activation='sigmoid'))
#(노드개수,입력변수 개수,활성화 함수)
print(model.summary())
"""
1. 학습용 데이터를 바탕으로 모델의 학습을 수행합니다.
    
step1. compile 메서드를 사용하여 최적화 모델 설정합니다.
       loss는 mean_squared_error, optimizer는 adam으로 설정합니다.
       
step2. fit 메서드를 사용하여 Dataset으로 변환된 학습용 데이터를 학습합니다.
       epochs는 100으로 설정합니다.
"""
model.compile(loss='mean_squared_error', optimizer='adam')
history = model.fit(train_ds, epochs=500, verbose=1)

"""
1. evaluate 메서드를 사용하여 테스트용 데이터의 loss 값을 계산합니다.
"""
loss = model.evaluate(test_X, test_Y, verbose=0)

"""
2. predict 메서드를 사용하여 테스트용 데이터의 예측값을 계산합니다.
"""
predictions = model.predict(test_X)

# 결과를 출력합니다.
print("테스트 데이터의 Loss 값: ", loss)
for i in range(5):
    print("%d 번째 테스트 데이터의 실제값: %f" % (i, test_Y.iloc[i]))
    print("%d 번째 테스트 데이터의 예측값: %f" % (i, predictions[i][0]))


원본 데이터 샘플 :
   Unnamed: 0     FB    TV  Newspaper  Sales
0           1  230.1  37.8       69.2   22.1
1           2   44.5  39.3       45.1   10.4
2           3   17.2  45.9       69.3    9.3
3           4  151.5  41.3       58.5   18.5
4           5  180.8  10.8       58.4   12.9 

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 10)                40        
                                                                 
 dense_11 (Dense)            (None, 1)                 11        
                                                                 
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/500
28/28 [==============================] - 1s 630us/step - loss: 1967.4781
Epoch 2/500
28/28 [==============================] - 0s 556us/step - loss: 405.0

Epoch 88/500
28/28 [==============================] - 0s 556us/step - loss: 3.8277
Epoch 89/500
28/28 [==============================] - 0s 556us/step - loss: 3.8159
Epoch 90/500
28/28 [==============================] - 0s 556us/step - loss: 3.8437
Epoch 91/500
28/28 [==============================] - 0s 556us/step - loss: 3.7624
Epoch 92/500
28/28 [==============================] - 0s 556us/step - loss: 3.7595
Epoch 93/500
28/28 [==============================] - 0s 556us/step - loss: 3.8510
Epoch 94/500
28/28 [==============================] - 0s 556us/step - loss: 3.7726
Epoch 95/500
28/28 [==============================] - 0s 556us/step - loss: 3.7770
Epoch 96/500
28/28 [==============================] - 0s 556us/step - loss: 3.6850
Epoch 97/500
28/28 [==============================] - 0s 556us/step - loss: 3.6950
Epoch 98/500
28/28 [==============================] - 0s 556us/step - loss: 3.7515
Epoch 99/500
28/28 [==============================] - 0s 556us/step - loss: 4.0509
Epoc

28/28 [==============================] - 0s 519us/step - loss: 3.4844
Epoch 186/500
28/28 [==============================] - 0s 519us/step - loss: 3.5085
Epoch 187/500
28/28 [==============================] - 0s 519us/step - loss: 3.5627
Epoch 188/500
28/28 [==============================] - 0s 519us/step - loss: 3.8454
Epoch 189/500
28/28 [==============================] - 0s 519us/step - loss: 3.7452
Epoch 190/500
28/28 [==============================] - 0s 519us/step - loss: 3.5486
Epoch 191/500
28/28 [==============================] - 0s 519us/step - loss: 3.5929
Epoch 192/500
28/28 [==============================] - 0s 556us/step - loss: 3.6803
Epoch 193/500
28/28 [==============================] - 0s 519us/step - loss: 4.2266
Epoch 194/500
28/28 [==============================] - 0s 556us/step - loss: 3.5389
Epoch 195/500
28/28 [==============================] - 0s 519us/step - loss: 3.7843
Epoch 196/500
28/28 [==============================] - 0s 519us/step - loss: 3.9952
Epoch 

28/28 [==============================] - 0s 556us/step - loss: 3.9808
Epoch 283/500
28/28 [==============================] - 0s 519us/step - loss: 5.4434
Epoch 284/500
28/28 [==============================] - 0s 556us/step - loss: 4.0214
Epoch 285/500
28/28 [==============================] - 0s 519us/step - loss: 3.8491
Epoch 286/500
28/28 [==============================] - 0s 556us/step - loss: 3.6572
Epoch 287/500
28/28 [==============================] - 0s 556us/step - loss: 4.2048
Epoch 288/500
28/28 [==============================] - 0s 556us/step - loss: 3.9069
Epoch 289/500
28/28 [==============================] - 0s 556us/step - loss: 3.8185
Epoch 290/500
28/28 [==============================] - 0s 556us/step - loss: 3.6741
Epoch 291/500
28/28 [==============================] - 0s 556us/step - loss: 3.6789
Epoch 292/500
28/28 [==============================] - 0s 519us/step - loss: 3.6047
Epoch 293/500
28/28 [==============================] - 0s 556us/step - loss: 3.7014
Epoch 

28/28 [==============================] - 0s 556us/step - loss: 3.4590
Epoch 380/500
28/28 [==============================] - 0s 556us/step - loss: 3.5638
Epoch 381/500
28/28 [==============================] - 0s 556us/step - loss: 3.8370
Epoch 382/500
28/28 [==============================] - 0s 556us/step - loss: 3.7143
Epoch 383/500
28/28 [==============================] - 0s 519us/step - loss: 3.8310
Epoch 384/500
28/28 [==============================] - 0s 556us/step - loss: 3.6102
Epoch 385/500
28/28 [==============================] - 0s 556us/step - loss: 3.7820
Epoch 386/500
28/28 [==============================] - 0s 556us/step - loss: 3.6772
Epoch 387/500
28/28 [==============================] - 0s 556us/step - loss: 3.5800
Epoch 388/500
28/28 [==============================] - 0s 556us/step - loss: 3.9003
Epoch 389/500
28/28 [==============================] - 0s 519us/step - loss: 4.0658
Epoch 390/500
28/28 [==============================] - 0s 556us/step - loss: 3.6190
Epoch 

28/28 [==============================] - 0s 556us/step - loss: 4.0090
Epoch 477/500
28/28 [==============================] - 0s 519us/step - loss: 3.8258
Epoch 478/500
28/28 [==============================] - 0s 519us/step - loss: 3.6122
Epoch 479/500
28/28 [==============================] - 0s 519us/step - loss: 3.7926
Epoch 480/500
28/28 [==============================] - 0s 556us/step - loss: 3.8841
Epoch 481/500
28/28 [==============================] - 0s 556us/step - loss: 4.2520
Epoch 482/500
28/28 [==============================] - 0s 556us/step - loss: 3.6545
Epoch 483/500
28/28 [==============================] - 0s 556us/step - loss: 3.9624
Epoch 484/500
28/28 [==============================] - 0s 519us/step - loss: 3.6169
Epoch 485/500
28/28 [==============================] - 0s 519us/step - loss: 3.8488
Epoch 486/500
28/28 [==============================] - 0s 519us/step - loss: 3.7788
Epoch 487/500
28/28 [==============================] - 0s 519us/step - loss: 3.6821
Epoch 

In [ ]:
"""X 데이터를 에포크를 100번으로 하여 학습합니다. 
verbose 인자는 학습 시, 화면에 출력되는 형태를 설정합니다. 
(0: 표기 없음, 1: 진행 바, 2: 에포크당 한 줄 출력)

evaluate() 메서드는 학습된 모델을 바탕으로 입력한 feature 데이터 X와 
label Y의 loss 값과 metrics 값을 출력합니다. 
이번 실습에서는 metrics 를 compile에서 설정하지 않았지만, 
분류에서는 일반적으로 accuracy를 사용하여 evaluate 사용 시, 
2개의 아웃풋을 리턴합니다
"""

In [12]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

np.random.seed(100)
tf.random.set_seed(100)

# sklearn에 저장된 데이터를 불러 옵니다.
X, Y = load_iris(return_X_y = True)

# DataFrame으로 변환
df = pd.DataFrame(X, columns=['꽃받침 길이','꽃받침 넓이', '꽃잎 길이', '꽃잎 넓이'])
df['클래스'] = Y

X = df.drop(columns=['클래스'])
Y = df['클래스']

# 학습용 평가용 데이터로 분리합니다
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.2, random_state = 42)

# Dataset 형태로 변환합니다.
train_ds = tf.data.Dataset.from_tensor_slices((train_X.values, train_Y))
train_ds = train_ds.shuffle(len(train_X)).batch(batch_size=5)

"""
1. keras를 활용하여 신경망 모델을 생성합니다.
   3가지 범주를 갖는 label 데이터를 분류하기 위해서 마지막 레이어 노드를 아래와 같이 설정합니다.
"""
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, input_dim=4),
    tf.keras.layers.Dense(3, activation='softmax')
    ])

# 학습용 데이터를 바탕으로 모델의 학습을 수행합니다.
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(train_ds, epochs=100, verbose=2)

# 테스트용 데이터를 바탕으로 학습된 모델을 평가합니다.
loss, acc = model.evaluate(test_X, test_Y)

# 테스트용 데이터의 예측값을 구합니다.
predictions = model.predict(test_X)

# 결과를 출력합니다.
print("테스트 데이터의 Accuracy 값: ", acc)
for i in range(5):
    print("%d 번째 테스트 데이터의 실제값: %d" % (i, test_Y.iloc[i]))
    print("%d 번째 테스트 데이터의 예측값: %d" % (i, np.argmax(predictions[i])))

Epoch 1/100
24/24 - 0s - loss: 1.3269 - accuracy: 0.6583 - 357ms/epoch - 15ms/step
Epoch 2/100
24/24 - 0s - loss: 0.9002 - accuracy: 0.6583 - 14ms/epoch - 584us/step
Epoch 3/100
24/24 - 0s - loss: 0.7204 - accuracy: 0.6583 - 14ms/epoch - 584us/step
Epoch 4/100
24/24 - 0s - loss: 0.6412 - accuracy: 0.6667 - 13ms/epoch - 542us/step
Epoch 5/100
24/24 - 0s - loss: 0.6124 - accuracy: 0.6750 - 13ms/epoch - 542us/step
Epoch 6/100
24/24 - 0s - loss: 0.5940 - accuracy: 0.6667 - 13ms/epoch - 542us/step
Epoch 7/100
24/24 - 0s - loss: 0.5729 - accuracy: 0.7167 - 14ms/epoch - 584us/step
Epoch 8/100
24/24 - 0s - loss: 0.5547 - accuracy: 0.7083 - 13ms/epoch - 542us/step
Epoch 9/100
24/24 - 0s - loss: 0.5384 - accuracy: 0.7250 - 14ms/epoch - 584us/step
Epoch 10/100
24/24 - 0s - loss: 0.5263 - accuracy: 0.7083 - 13ms/epoch - 542us/step
Epoch 11/100
24/24 - 0s - loss: 0.5114 - accuracy: 0.7583 - 13ms/epoch - 542us/step
Epoch 12/100
24/24 - 0s - loss: 0.5011 - accuracy: 0.7333 - 13ms/epoch - 542us/step
E

Epoch 99/100
24/24 - 0s - loss: 0.1171 - accuracy: 0.9833 - 13ms/epoch - 542us/step
Epoch 100/100
24/24 - 0s - loss: 0.1149 - accuracy: 0.9667 - 13ms/epoch - 542us/step
1/1 [==============================] - 0s 34ms/step
테스트 데이터의 Accuracy 값:  0.9666666388511658
0 번째 테스트 데이터의 실제값: 1
0 번째 테스트 데이터의 예측값: 1
1 번째 테스트 데이터의 실제값: 0
1 번째 테스트 데이터의 예측값: 0
2 번째 테스트 데이터의 실제값: 2
2 번째 테스트 데이터의 예측값: 2
3 번째 테스트 데이터의 실제값: 1
3 번째 테스트 데이터의 예측값: 1
4 번째 테스트 데이터의 실제값: 1
4 번째 테스트 데이터의 예측값: 1
